avarage code for prep_guru

In [ ]:
import asyncio
import os
import io
import time
import pickle
from dotenv import load_dotenv
import azure.cognitiveservices.speech as speechsdk
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
import PyPDF2

# Load environment variables
load_dotenv()

class LanguageModelProcessor:
    def __init__(self):
        self.llm = ChatGroq(
            temperature=0,
            model_name="llama-3.1-70b-versatile",
            groq_api_key=os.getenv("GROQ_API_KEY")
        )
        self.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        
        with open('system_prompt2.txt', 'r') as file:
            system_prompt = file.read().strip()
        
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            HumanMessagePromptTemplate.from_template("{text}")
        ])
        
        self.conversation = LLMChain(
            llm=self.llm,
            prompt=self.prompt,
            memory=self.memory
        )

    def process(self, text):
        self.memory.chat_memory.add_user_message(text)
        start_time = time.time()
        response = self.conversation.invoke({"text": text})
        end_time = time.time()
        self.memory.chat_memory.add_ai_message(response['text'])
        elapsed_time = int((end_time - start_time) * 1000)
        print(f"LLM ({elapsed_time}ms): {response['text']}")
        return response['text']

    def load_resume(self, resume_path):
        with open(resume_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            resume_text = ""
            for page in reader.pages:
                resume_text += page.extract_text()
        return resume_text.strip()

    def save_memory(self, filepath='conversation_memory.pkl'):
        with open(filepath, 'wb') as f:
            pickle.dump(self.memory, f)
        print(f"Memory saved to {filepath}")

    def load_memory(self, filepath='conversation_memory.pkl'):
        if os.path.exists(filepath):
            with open(filepath, 'rb') as f:
                self.memory = pickle.load(f)
            print(f"Memory loaded from {filepath}")
        else:
            print(f"No memory file found at {filepath}, starting fresh.")

    def save_conversation(self, user_input, ai_response, filepath='conversation_log.txt'):
        try:
            with open(filepath, 'a', encoding='utf-8') as log_file:
                log_file.write(f"User: {user_input}\nAI: {ai_response}\n\n")
        except UnicodeEncodeError as e:
            print(f"UnicodeEncodeError: {e}")

    def load_conversation(self, filepath='conversation_log.txt'):
        if os.path.exists(filepath):
            with open(filepath, 'r', encoding='utf-8') as log_file:
                previous_conversation = log_file.read()
            print("Previous conversation loaded:\n", previous_conversation)
            return True
        else:
            print("Unable to load conversation_log.txt. I don't have your last information available.")
            return False


def setup_azure_speech(api_key, region):
    speech_config = speechsdk.SpeechConfig(subscription=api_key, region=region)
    speech_config.speech_recognition_language = "en-IN"
    speech_config.set_property(speechsdk.PropertyId.SpeechServiceConnection_InitialSilenceTimeoutMs, '5000')
    speech_config.set_property(speechsdk.PropertyId.SpeechServiceConnection_EndSilenceTimeoutMs, '3000')

    audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)
    return speech_recognizer


def setup_azure_tts(api_key, region):
    speech_config = speechsdk.SpeechConfig(subscription=api_key, region=region)
    audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)
    speech_config.speech_synthesis_voice_name = 'en-IN-AaravNeural'
    speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)
    return speech_synthesizer


def text_to_speech(text, speech_synthesizer):
    ssml_text = f"""
    <speak version='1.0' xmlns='http://www.w3.org/2001/10/synthesis' xml:lang='en-IN'>
        <voice name='en-IN-AaravNeural'>
            <prosody rate='medium'>
                {text}
            </prosody>
        </voice>
    </speak>
    """
    try:
        speech_synthesis_result = speech_synthesizer.speak_ssml_async(ssml_text).get()
        if speech_synthesis_result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
            print("Speech synthesis complete")
        else:
            print(f"Speech synthesis failed: {speech_synthesis_result.reason}")
    except Exception as e:
        print(f"An error occurred during speech synthesis: {e}")


class ConversationManager:
    def __init__(self):
        self.llm = LanguageModelProcessor()
        self.api_key = os.getenv("AZURE_API_KEY")
        self.region = os.getenv("AZURE_REGION")
        self.speech_recognizer = setup_azure_speech(self.api_key, self.region)
        self.speech_synthesizer = setup_azure_tts(self.api_key, self.region)
        self.is_running = True

    async def listen_for_speech(self):
        print("Listening...")
        result = await self.recognize_speech()
        if result.reason == speechsdk.ResultReason.RecognizedSpeech:
            return result.text
        elif result.reason == speechsdk.ResultReason.NoMatch:
            print("No speech could be recognized")
        return None

    async def recognize_speech(self):
        loop = asyncio.get_running_loop()
        future = loop.create_future()

        def recognized_cb(evt):
            if not future.done():
                loop.call_soon_threadsafe(future.set_result, evt.result)

        def canceled_cb(evt):
            if not future.done():
                loop.call_soon_threadsafe(future.set_exception, Exception(f"Speech recognition canceled: {evt.reason}"))

        self.speech_recognizer.recognized.connect(recognized_cb)
        self.speech_recognizer.canceled.connect(canceled_cb)

        await loop.run_in_executor(None, self.speech_recognizer.start_continuous_recognition)

        try:
            result = await asyncio.wait_for(future, timeout=180.0)  # 180 second timeout
        except asyncio.TimeoutError:
            print("Speech recognition timed out")
            result = None
        finally:
            await loop.run_in_executor(None, self.speech_recognizer.stop_continuous_recognition)

        return result

    async def ask_for_introduction(self):
        # First introduction
        intro_prompt = "Please introduce yourself."
        print(f"AI: {intro_prompt}")
        text_to_speech(intro_prompt, self.speech_synthesizer)

        user_intro1 = await self.listen_for_speech()
        if user_intro1:
            print(f"User: {user_intro1}")
            feedback_intro1 = self.llm.process(user_intro1)
            print(f"AI: {feedback_intro1}")
            text_to_speech(feedback_intro1, self.speech_synthesizer)

        # Second introduction
        intro_prompt_2 = "Please introduce yourself again, sharing more details."
        print(f"AI: {intro_prompt_2}")
        text_to_speech(intro_prompt_2, self.speech_synthesizer)

        user_intro2 = await self.listen_for_speech()
        if user_intro2:
            print(f"User: {user_intro2}")
            feedback_intro2 = self.llm.process(user_intro2)
            print(f"AI: {feedback_intro2}")
            text_to_speech(feedback_intro2, self.speech_synthesizer)

    async def ask_questions_based_on_resume(self, resume_text):
        questions_prompt = f"Based on the following resume, please ask relevant questions: {resume_text}"
        questions = self.llm.process(questions_prompt)
        print(f"AI: {questions}")
        text_to_speech(questions, self.speech_synthesizer)

        while True:
            user_response = await self.listen_for_speech()
            if user_response:
                print(f"User: {user_response}")
                feedback = self.llm.process(user_response)
                print(f"AI: {feedback}")
                text_to_speech(feedback, self.speech_synthesizer)

    async def main(self):
        # Start with the welcome message
        welcome_message = "I am GetScreened PrepGuru. I will guide you step by step to improve your interview skills and help you become a better version of yourself."
        print(f"AI: {welcome_message}")
        text_to_speech(welcome_message, self.speech_synthesizer)

        # Ask for the introduction twice
        await self.ask_for_introduction()

        # Load resume and process it after introductions
        resume_path = r"Resume\Ankush Mahore.pdf"  # Adjust this path to your resume file
        resume_text = self.llm.load_resume(resume_path)

        # Ask questions based on the resume
        await self.ask_questions_based_on_resume(resume_text)

if __name__ == "__main__":
    manager = ConversationManager()
    asyncio.run(manager.main())


this code ask quation best way on resume  

In [ ]:
import asyncio
import os
import io
import time
import pickle
from dotenv import load_dotenv
import azure.cognitiveservices.speech as speechsdk
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
import PyPDF2

# Load environment variables
load_dotenv()

class LanguageModelProcessor:
    def __init__(self):
        self.llm = ChatGroq(
            temperature=0,
            model_name="llama-3.1-70b-versatile",
            groq_api_key=os.getenv("GROQ_API_KEY")
        )
        self.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        
        with open('system_prompt2.txt', 'r') as file:
            system_prompt = file.read().strip()
        
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            HumanMessagePromptTemplate.from_template("{text}")
        ])
        
        self.conversation = LLMChain(
            llm=self.llm,
            prompt=self.prompt,
            memory=self.memory
        )

    def process(self, text):
        self.memory.chat_memory.add_user_message(text)
        start_time = time.time()
        response = self.conversation.invoke({"text": text})
        end_time = time.time()
        self.memory.chat_memory.add_ai_message(response['text'])
        elapsed_time = int((end_time - start_time) * 1000)
        print(f"LLM ({elapsed_time}ms): {response['text']}")
        return response['text']

    def load_resume(self, resume_path):
        with open(resume_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            resume_text = ""
            for page in reader.pages:
                resume_text += page.extract_text()
        return resume_text.strip()

    def save_memory(self, filepath='conversation_memory.pkl'):
        with open(filepath, 'wb') as f:
            pickle.dump(self.memory, f)
        print(f"Memory saved to {filepath}")

    def load_memory(self, filepath='conversation_memory.pkl'):
        if os.path.exists(filepath):
            with open(filepath, 'rb') as f:
                self.memory = pickle.load(f)
            print(f"Memory loaded from {filepath}")
        else:
            print(f"No memory file found at {filepath}, starting fresh.")

    def save_conversation(self, user_input, ai_response, filepath='conversation_log.txt'):
        try:
            with open(filepath, 'a', encoding='utf-8') as log_file:
                log_file.write(f"User: {user_input}\nAI: {ai_response}\n\n")
        except UnicodeEncodeError as e:
            print(f"UnicodeEncodeError: {e}")

    def load_conversation(self, filepath='conversation_log.txt'):
        if os.path.exists(filepath):
            with open(filepath, 'r', encoding='utf-8') as log_file:
                previous_conversation = log_file.read()
            print("Previous conversation loaded:\n", previous_conversation)
            return True
        else:
            print("Unable to load conversation_log.txt. I don't have your last information available.")
            return False


def setup_azure_speech(api_key, region):
    speech_config = speechsdk.SpeechConfig(subscription=api_key, region=region)
    speech_config.speech_recognition_language = "en-IN"
    speech_config.set_property(speechsdk.PropertyId.SpeechServiceConnection_InitialSilenceTimeoutMs, '5000')
    speech_config.set_property(speechsdk.PropertyId.SpeechServiceConnection_EndSilenceTimeoutMs, '3000')

    audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)
    return speech_recognizer


def setup_azure_tts(api_key, region):
    speech_config = speechsdk.SpeechConfig(subscription=api_key, region=region)
    audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)
    speech_config.speech_synthesis_voice_name = 'en-IN-AaravNeural'
    speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)
    return speech_synthesizer


def text_to_speech(text, speech_synthesizer):
    ssml_text = f"""
    <speak version='1.0' xmlns='http://www.w3.org/2001/10/synthesis' xml:lang='en-IN'>
        <voice name='en-IN-AaravNeural'>
            <prosody rate='medium'>
                {text}
            </prosody>
        </voice>
    </speak>
    """
    try:
        speech_synthesis_result = speech_synthesizer.speak_ssml_async(ssml_text).get()
        if speech_synthesis_result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
            print("Speech synthesis complete")
        else:
            print(f"Speech synthesis failed: {speech_synthesis_result.reason}")
    except Exception as e:
        print(f"An error occurred during speech synthesis: {e}")


class ConversationManager:
    def __init__(self):
        self.llm = LanguageModelProcessor()
        self.api_key = os.getenv("AZURE_API_KEY")
        self.region = os.getenv("AZURE_REGION")
        self.speech_recognizer = setup_azure_speech(self.api_key, self.region)
        self.speech_synthesizer = setup_azure_tts(self.api_key, self.region)
        self.is_running = True

    async def listen_for_speech(self):
        print("Listening...")
        result = await self.recognize_speech()
        if result.reason == speechsdk.ResultReason.RecognizedSpeech:
            return result.text
        elif result.reason == speechsdk.ResultReason.NoMatch:
            print("No speech could be recognized")
        return None

    async def recognize_speech(self):
        loop = asyncio.get_running_loop()
        future = loop.create_future()

        def recognized_cb(evt):
            if not future.done():
                loop.call_soon_threadsafe(future.set_result, evt.result)

        def canceled_cb(evt):
            if not future.done():
                loop.call_soon_threadsafe(future.set_exception, Exception(f"Speech recognition canceled: {evt.reason}"))

        self.speech_recognizer.recognized.connect(recognized_cb)
        self.speech_recognizer.canceled.connect(canceled_cb)

        await loop.run_in_executor(None, self.speech_recognizer.start_continuous_recognition)

        try:
            result = await asyncio.wait_for(future, timeout=180.0)  # 180 second timeout
        except asyncio.TimeoutError:
            print("Speech recognition timed out")
            result = None
        finally:
            await loop.run_in_executor(None, self.speech_recognizer.stop_continuous_recognition)

        return result

    async def ask_for_introduction(self):
        iteration_count = 0
        feedback = ""

        while "good" not in feedback.lower() and iteration_count < 2:
            # First introduction
            intro_prompt = "Please introduce yourself."
            print(f"AI: {intro_prompt}")
            text_to_speech(intro_prompt, self.speech_synthesizer)

            user_intro = await self.listen_for_speech()
            if user_intro:
                print(f"User: {user_intro}")
                feedback = self.llm.process(user_intro)
                print(f"AI: {feedback}")
                text_to_speech(feedback, self.speech_synthesizer)

            iteration_count += 1

        # After 2 iterations, suggest practice and move on
        if "good" not in feedback.lower():
            text_to_speech("Please practice this introduction more. Let's move on to the next question.", self.speech_synthesizer)


    async def ask_questions_based_on_resume(self, resume_text):
        questions_prompt = f"Based on the following resume, please ask relevant questions: {resume_text}"
        questions = self.llm.process(questions_prompt)
        print(f"AI: {questions}")
        text_to_speech(questions, self.speech_synthesizer)

        while True:
            user_response = await self.listen_for_speech()
            if user_response:
                print(f"User: {user_response}")
                feedback = self.llm.process(user_response)
                print(f"AI: {feedback}")
                text_to_speech(feedback, self.speech_synthesizer)

    async def main(self):
        # Start with the welcome message
        welcome_message = "I am GetScreened PrepGuru. I will guide you step by step to improve your interview skills and help you become a better version of yourself."
        print(f"AI: {welcome_message}")
        text_to_speech(welcome_message, self.speech_synthesizer)

        # Ask for the introduction twice
        await self.ask_for_introduction()

        # Load resume and process it after introductions
        resume_path = r"Resume\Ankush Mahore.pdf"  # Adjust this path to your resume file
        resume_text = self.llm.load_resume(resume_path)

        # Ask questions based on the resume
        await self.ask_questions_based_on_resume(resume_text)

if __name__ == "__main__":
    manager = ConversationManager()
    asyncio.run(manager.main())


This code make mistake in introducatin part

In [ ]:
import asyncio
import os
import io
import time
import pickle
from dotenv import load_dotenv
import azure.cognitiveservices.speech as speechsdk
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
import PyPDF2

# Load environment variables
load_dotenv()

class LanguageModelProcessor:
    def __init__(self):
        self.llm = ChatGroq(
            temperature=0,
            model_name="llama-3.1-70b-versatile",
            groq_api_key=os.getenv("GROQ_API_KEY")
        )
        self.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        
        with open('system_prompt2.txt', 'r') as file:
            system_prompt = file.read().strip()
        
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            HumanMessagePromptTemplate.from_template("{text}")
        ])
        
        self.conversation = LLMChain(
            llm=self.llm,
            prompt=self.prompt,
            memory=self.memory
        )

    def process(self, text):
        self.memory.chat_memory.add_user_message(text)
        start_time = time.time()
        response = self.conversation.invoke({"text": text})
        end_time = time.time()
        self.memory.chat_memory.add_ai_message(response['text'])
        elapsed_time = int((end_time - start_time) * 1000)
        print(f"LLM ({elapsed_time}ms): {response['text']}")
        return response['text']

    def load_resume(self, resume_path):
        with open(resume_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            resume_text = ""
            for page in reader.pages:
                resume_text += page.extract_text()
        return resume_text.strip()

    def save_memory(self, filepath='conversation_memory.pkl'):
        with open(filepath, 'wb') as f:
            pickle.dump(self.memory, f)
        print(f"Memory saved to {filepath}")

    def load_memory(self, filepath='conversation_memory.pkl'):
        if os.path.exists(filepath):
            with open(filepath, 'rb') as f:
                self.memory = pickle.load(f)
            print(f"Memory loaded from {filepath}")
        else:
            print(f"No memory file found at {filepath}, starting fresh.")

    def save_conversation(self, user_input, ai_response, filepath='conversation_log.txt'):
        try:
            with open(filepath, 'a', encoding='utf-8') as log_file:
                log_file.write(f"User: {user_input}\nAI: {ai_response}\n\n")
        except UnicodeEncodeError as e:
            print(f"UnicodeEncodeError: {e}")

    def load_conversation(self, filepath='conversation_log.txt'):
        if os.path.exists(filepath):
            with open(filepath, 'r', encoding='utf-8') as log_file:
                previous_conversation = log_file.read()
            print("Previous conversation loaded:\n", previous_conversation)
            return True
        else:
            print("Unable to load conversation_log.txt. I don't have your last information available.")
            return False


def setup_azure_speech(api_key, region):
    speech_config = speechsdk.SpeechConfig(subscription=api_key, region=region)
    speech_config.speech_recognition_language = "en-IN"
    speech_config.set_property(speechsdk.PropertyId.SpeechServiceConnection_InitialSilenceTimeoutMs, '5000')
    speech_config.set_property(speechsdk.PropertyId.SpeechServiceConnection_EndSilenceTimeoutMs, '3000')

    audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)
    return speech_recognizer


def setup_azure_tts(api_key, region):
    speech_config = speechsdk.SpeechConfig(subscription=api_key, region=region)
    audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)
    speech_config.speech_synthesis_voice_name = 'en-IN-AaravNeural'
    speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)
    return speech_synthesizer


def text_to_speech(text, speech_synthesizer):
    ssml_text = f"""
    <speak version='1.0' xmlns='http://www.w3.org/2001/10/synthesis' xml:lang='en-IN'>
        <voice name='en-IN-AaravNeural'>
            <prosody rate='medium'>
                {text}
            </prosody>
        </voice>
    </speak>
    """
    try:
        speech_synthesis_result = speech_synthesizer.speak_ssml_async(ssml_text).get()
        if speech_synthesis_result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
            print("Speech synthesis complete")
        else:
            print(f"Speech synthesis failed: {speech_synthesis_result.reason}")
    except Exception as e:
        print(f"An error occurred during speech synthesis: {e}")


class ConversationManager:
    def __init__(self):
        self.llm = LanguageModelProcessor()
        self.api_key = os.getenv("AZURE_API_KEY")
        self.region = os.getenv("AZURE_REGION")
        self.speech_recognizer = setup_azure_speech(self.api_key, self.region)
        self.speech_synthesizer = setup_azure_tts(self.api_key, self.region)
        self.is_running = True

    async def listen_for_speech(self):
        print("Listening...")
        result = await self.recognize_speech()
        if result.reason == speechsdk.ResultReason.RecognizedSpeech:
            return result.text
        elif result.reason == speechsdk.ResultReason.NoMatch:
            print("No speech could be recognized")
        return None

    async def recognize_speech(self):
        loop = asyncio.get_running_loop()
        future = loop.create_future()

        def recognized_cb(evt):
            if not future.done():
                loop.call_soon_threadsafe(future.set_result, evt.result)

        def canceled_cb(evt):
            if not future.done():
                loop.call_soon_threadsafe(future.set_exception, Exception(f"Speech recognition canceled: {evt.reason}"))

        self.speech_recognizer.recognized.connect(recognized_cb)
        self.speech_recognizer.canceled.connect(canceled_cb)

        await loop.run_in_executor(None, self.speech_recognizer.start_continuous_recognition)

        try:
            result = await asyncio.wait_for(future, timeout=180.0)  # 180 second timeout
        except asyncio.TimeoutError:
            print("Speech recognition timed out")
            result = None
        finally:
            await loop.run_in_executor(None, self.speech_recognizer.stop_continuous_recognition)

        return result

    # async def ask_for_introduction(self):
    #     iteration_count = 0
    #     feedback = ""

    #     while "good" not in feedback.lower() and iteration_count < 2:
    #         # First introduction
    #         intro_prompt = "Please introduce yourself."
    #         print(f"AI: {intro_prompt}")
    #         text_to_speech(intro_prompt, self.speech_synthesizer)

    #         user_intro = await self.listen_for_speech()
    #         if user_intro:
    #             print(f"User: {user_intro}")
    #             feedback = self.llm.process(user_intro)
    #             print(f"AI: {feedback}")
    #             text_to_speech(feedback, self.speech_synthesizer)

    #         iteration_count += 1

    
    # async def ask_for_introduction(self):
    #     iteration_count = 0  # To count the number of attempts

    # while iteration_count < 3:  # Allow for up to 3 attempts
    #     # Step 1: Ask for introduction
    #     intro_prompt = "Please introduce yourself."
    #     print(f"AI: {intro_prompt}")
    #     text_to_speech(intro_prompt, self.speech_synthesizer)

    #     # Capture user's introduction
    #     user_intro = await self.listen_for_speech()
    #     if user_intro:
    #         print(f"User: {user_intro}")

    #         # Step 2: Analyze user's introduction and give feedback
    #         feedback = self.llm.process(f"Here is an introduction: {user_intro}. Give me feedback on the introduction, what is good, and what is missing.")
    #         print(f"AI Feedback: {feedback}")
    #         text_to_speech(feedback, self.speech_synthesizer)

    #         # Step 3: Check if feedback is satisfactory
    #         if "good" in feedback.lower():
    #             print("Feedback was satisfactory.")
    #             break  # Exit loop if feedback is satisfactory
    #         else:
    #             print("Your introduction needs improvement.")
    #             text_to_speech("Your introduction needs improvement. Please try again.", self.speech_synthesizer)
    #     else:
    #         print("No valid introduction received. Please try again.")
    #         text_to_speech("No valid introduction received. Please try again.", self.speech_synthesizer)

    #     iteration_count += 1  # Increment the attempt count

    # # After 3 attempts, provide final feedback
    # if iteration_count == 3:
    #     practice_feedback = "You have made three attempts. Please practice your introduction more."
    #     print(practice_feedback)
    #     text_to_speech(practice_feedback, self.speech_synthesizer)

################################################################
    async def ask_for_introduction(self):
        iteration_count = 0  # To count the number of attempts

        while iteration_count < 3:  # Allow for up to 3 attempts
            # Step 1: Ask for introduction
            intro_prompt = "Please introduce yourself."
            print(f"AI: {intro_prompt}")
            text_to_speech(intro_prompt, self.speech_synthesizer)

            # Capture user's introduction
            user_intro = await self.listen_for_speech()
            if user_intro:
                print(f"User: {user_intro}")

                # Step 2: Analyze user's introduction and give feedback
                feedback = self.llm.process(f"Here is an introduction: {user_intro}. Give me feedback on the introduction, what is good, and what is missing.")
                print(f"AI Feedback: {feedback}")
                text_to_speech(feedback, self.speech_synthesizer)

                # Step 3: Check if feedback is satisfactory
                if "good" in feedback.lower():
                    print("Feedback was satisfactory.")
                    break  # Exit loop if feedback is satisfactory
                else:
                    print("Your introduction needs improvement.")
                    text_to_speech("Your introduction needs improvement. Please try again.", self.speech_synthesizer)
            else:
                print("No valid introduction received. Please try again.")
                text_to_speech("No valid introduction received. Please try again.", self.speech_synthesizer)

            iteration_count += 1  # Increment the attempt count

        # After 3 attempts, provide final feedback
        if iteration_count == 3:
            practice_feedback = "You have made three attempts. Please practice your introduction more."
            print(practice_feedback)
            text_to_speech(practice_feedback, self.speech_synthesizer)

        # Optionally, you could restart asking for introduction or move to the next stage
        # Uncomment the following line if you want to ask again after three attempts
        # await self.ask_for_introduction()  # Restart the process if needed





###############################################################
        # Optionally, you could restart asking for introduction or move to the next stage
        # Uncomment the following line if you want to ask again after three attempts
        # await self.ask_for_introduction()  # Restart the process if needed



        # After 2 iterations, suggest practice and move on
        # if "good" not in feedback.lower():
        #     text_to_speech("Please practice this introduction more. Let's move on to the next question.", self.speech_synthesizer)


    async def ask_questions_based_on_resume(self, resume_text):
        questions_prompt = f"Based on the following resume, please ask relevant questions: {resume_text}"
        questions = self.llm.process(questions_prompt)
        print(f"AI: {questions}")
        text_to_speech(questions, self.speech_synthesizer)

        while True:
            user_response = await self.listen_for_speech()
            if user_response:
                print(f"User: {user_response}")
                feedback = self.llm.process(user_response)
                print(f"AI: {feedback}")
                text_to_speech(feedback, self.speech_synthesizer)

    async def main(self):
        # Start with the welcome message
        welcome_message = "I am GetScreened PrepGuru. I will guide you step by step to improve your interview skills and help you become a better version of yourself."
        print(f"AI: {welcome_message}")
        text_to_speech(welcome_message, self.speech_synthesizer)

        # Ask for the introduction twice
        await self.ask_for_introduction()

        # Load resume and process it after introductions
        resume_path = r"Resume\Ankush Mahore.pdf"  # Adjust this path to your resume file
        resume_text = self.llm.load_resume(resume_path)

        # Ask questions based on the resume
        await self.ask_questions_based_on_resume(resume_text)

if __name__ == "__main__":
    manager = ConversationManager()
    asyncio.run(manager.main())


introducation code

In [ ]:
import asyncio
import os
import io
import time
import pickle
from dotenv import load_dotenv
import azure.cognitiveservices.speech as speechsdk
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
import PyPDF2

# Load environment variables
load_dotenv()

class LanguageModelProcessor:
    def __init__(self):
        self.llm = ChatGroq(
            temperature=0,
            model_name="llama-3.1-70b-versatile",
            groq_api_key=os.getenv("GROQ_API_KEY")
        )
        self.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        
        with open('system_prompt2.txt', 'r') as file:
            system_prompt = file.read().strip()
        
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            HumanMessagePromptTemplate.from_template("{text}")
        ])
        
        self.conversation = LLMChain(
            llm=self.llm,
            prompt=self.prompt,
            memory=self.memory
        )

    def process(self, text):
        self.memory.chat_memory.add_user_message(text)
        start_time = time.time()
        response = self.conversation.invoke({"text": text})
        end_time = time.time()
        self.memory.chat_memory.add_ai_message(response['text'])
        elapsed_time = int((end_time - start_time) * 1000)
        print(f"LLM ({elapsed_time}ms): {response['text']}")
        return response['text']

    def load_resume(self, resume_path):
        with open(resume_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            resume_text = ""
            for page in reader.pages:
                resume_text += page.extract_text()
        return resume_text.strip()

    def save_memory(self, filepath='conversation_memory.pkl'):
        with open(filepath, 'wb') as f:
            pickle.dump(self.memory, f)
        print(f"Memory saved to {filepath}")

    def load_memory(self, filepath='conversation_memory.pkl'):
        if os.path.exists(filepath):
            with open(filepath, 'rb') as f:
                self.memory = pickle.load(f)
            print(f"Memory loaded from {filepath}")
        else:
            print(f"No memory file found at {filepath}, starting fresh.")

    def save_conversation(self, user_input, ai_response, filepath='conversation_log.txt'):
        try:
            with open(filepath, 'a', encoding='utf-8') as log_file:
                log_file.write(f"User: {user_input}\nAI: {ai_response}\n\n")
        except UnicodeEncodeError as e:
            print(f"UnicodeEncodeError: {e}")

    def load_conversation(self, filepath='conversation_log.txt'):
        if os.path.exists(filepath):
            with open(filepath, 'r', encoding='utf-8') as log_file:
                previous_conversation = log_file.read()
            print("Previous conversation loaded:\n", previous_conversation)
            return True
        else:
            print("Unable to load conversation_log.txt. I don't have your last information available.")
            return False


def setup_azure_speech(api_key, region):
    speech_config = speechsdk.SpeechConfig(subscription=api_key, region=region)
    speech_config.speech_recognition_language = "en-IN"
    speech_config.set_property(speechsdk.PropertyId.SpeechServiceConnection_InitialSilenceTimeoutMs, '5000')
    speech_config.set_property(speechsdk.PropertyId.SpeechServiceConnection_EndSilenceTimeoutMs, '4000')

    audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)
    return speech_recognizer


def setup_azure_tts(api_key, region):
    speech_config = speechsdk.SpeechConfig(subscription=api_key, region=region)
    audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)
    speech_config.speech_synthesis_voice_name = 'en-IN-AaravNeural'
    speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)
    return speech_synthesizer


def text_to_speech(text, speech_synthesizer):
    ssml_text = f"""
    <speak version='1.0' xmlns='http://www.w3.org/2001/10/synthesis' xml:lang='en-IN'>
        <voice name='en-IN-AaravNeural'>
            <prosody rate='medium'>
                {text}
            </prosody>
        </voice>
    </speak>
    """
    try:
        speech_synthesis_result = speech_synthesizer.speak_ssml_async(ssml_text).get()
        if speech_synthesis_result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
            print("Speech synthesis complete")
        else:
            print(f"Speech synthesis failed: {speech_synthesis_result.reason}")
    except Exception as e:
        print(f"An error occurred during speech synthesis: {e}")


class ConversationManager:
    def __init__(self):
        self.llm = LanguageModelProcessor()
        self.api_key = os.getenv("AZURE_API_KEY")
        self.region = os.getenv("AZURE_REGION")
        self.speech_recognizer = setup_azure_speech(self.api_key, self.region)
        self.speech_synthesizer = setup_azure_tts(self.api_key, self.region)
        self.is_running = True

    async def listen_for_speech(self):
        print("Listening...")
        result = await self.recognize_speech()
        if result.reason == speechsdk.ResultReason.RecognizedSpeech:
            return result.text
        elif result.reason == speechsdk.ResultReason.NoMatch:
            print("No speech could be recognized")
        return None

    async def recognize_speech(self):
        loop = asyncio.get_running_loop()
        future = loop.create_future()

        def recognized_cb(evt):
            if not future.done():
                loop.call_soon_threadsafe(future.set_result, evt.result)

        def canceled_cb(evt):
            if not future.done():
                loop.call_soon_threadsafe(future.set_exception, Exception(f"Speech recognition canceled: {evt.reason}"))

        self.speech_recognizer.recognized.connect(recognized_cb)
        self.speech_recognizer.canceled.connect(canceled_cb)

        await loop.run_in_executor(None, self.speech_recognizer.start_continuous_recognition)

        try:
            result = await asyncio.wait_for(future, timeout=180.0)  # 180 second timeout
        except asyncio.TimeoutError:
            print("Speech recognition timed out")
            result = None
        finally:
            await loop.run_in_executor(None, self.speech_recognizer.stop_continuous_recognition)

        return result

    async def ask_for_introduction(self):
        iteration_count = 0
        feedback = ""

        while iteration_count < 3:  # Allow two attempts for the introduction
            intro_prompt = "Please introduce yourself."
            print(f"AI: {intro_prompt}")
            text_to_speech(intro_prompt, self.speech_synthesizer)

            user_intro = await self.listen_for_speech()
            if user_intro:
                print(f"User: {user_intro}")
                feedback = self.llm.process(user_intro)
                print(f"AI: {feedback}")
                text_to_speech(feedback, self.speech_synthesizer)

                # Suggest improvement if the introduction isn't good
                if "good" not in feedback.lower():
                    improvement_suggestion = (
                        "Consider adding more details about your key skills, "
                        "specific experiences related to the job, and your motivations "
                        "for applying to this company."
                    )
                    print(f"AI: {improvement_suggestion}")
                    text_to_speech(improvement_suggestion, self.speech_synthesizer)

            iteration_count += 1

        # After two iterations, give final encouragement
        final_encouragement = "Thank you for your introductions. Let's move on to the next question."
        print(f"AI: {final_encouragement}")
        text_to_speech(final_encouragement, self.speech_synthesizer)

    async def main(self):
        # Start with the welcome message
        welcome_message = "I am GetScreened PrepGuru. I will guide you step by step to improve your interview skills and help you become a better version of yourself."
        print(f"AI: {welcome_message}")
        text_to_speech(welcome_message, self.speech_synthesizer)

        # Ask for the introduction
        await self.ask_for_introduction()

        # Load resume and process it after introductions
        resume_path = r"Resume\Ankush Mahore.pdf"  # Adjust this path to your resume file
        resume_text = self.llm.load_resume(resume_path)

        # Ask questions based on the resume (if needed)
        # await self.ask_questions_based_on_resume(resume_text)  # Uncomment if required


if __name__ == "__main__":
    manager = ConversationManager()
    asyncio.run(manager.main())


Introducation

import os
import time
import asyncio
import pickle
import PyPDF2
from pydub import AudioSegment
from pydub.playback import play
from dotenv import load_dotenv
from deepgram import DeepgramClient, SpeakOptions, LiveTranscriptionEvents, DeepgramClientOptions, LiveOptions, Microphone
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain

# Load environment variables
load_dotenv()

def text_to_speech(text):
    """Convert text to speech using the Deepgram API."""
    try:
        deepgram = DeepgramClient(api_key=os.getenv("DEEPGRAM_API_KEY"))
        url = "https://api.deepgram.com/v1/speak?model=aura-hera-en"
        api_key = os.getenv("DEEPGRAM_API_KEY")
        headers = {"Authorization": f"Token {api_key}", "Content-Type": "application/json"}
        options = SpeakOptions(
            model="aura-zeus-en",
            encoding="linear16",
            container="wav"
        )
        
        SPEAK_OPTIONS = {"text": text}
        filename = "output.wav"
        response = deepgram.speak.v("1").save(filename, SPEAK_OPTIONS, options)
        audio = AudioSegment.from_wav(filename)
        play(audio)
    except Exception as e:
        print(f"Error occurred: {e}")

class LanguageModelProcessor:
    def __init__(self):
        """Initialize the language model processor."""
        self.llm = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768", groq_api_key=os.getenv("GROQ_API_KEY"))
        self.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        
        # Load the system prompt from a file
        with open('system_prompt2.txt', 'r') as file:
            system_prompt = file.read().strip()
        
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            HumanMessagePromptTemplate.from_template("{text}")
        ])
        
        self.conversation = LLMChain(
            llm=self.llm,
            prompt=self.prompt,
            memory=self.memory
        )

    def provide_feedback_on_introduction(self, introduction_text):
        """Analyze the introduction text and provide feedback on its effectiveness."""
        feedback_prompt = f"Analyze this introduction: '{introduction_text}'. Provide feedback on its effectiveness."
        response = self.conversation.invoke({"text": feedback_prompt})
        self.save_conversation(feedback_prompt, response['text'])  # Save to conversation log
        return response['text']

    def process(self, text):
        """Process the user input text and get a response from the LLM."""
        self.memory.chat_memory.add_user_message(text)  # Add user message to memory
        start_time = time.time()
        # Get the response from the LLM
        response = self.conversation.invoke({"text": text})
        end_time = time.time()
        self.memory.chat_memory.add_ai_message(response['text'])  # Add AI response to memory
        elapsed_time = int((end_time - start_time) * 1000)
        print(f"LLM ({elapsed_time}ms): {response['text']}")
        self.save_conversation(text, response['text'])  # Save to conversation log
        return response['text']

    def save_conversation(self, user_input, ai_response, filepath='conversation_log.txt'):
        """Save the conversation between user and AI to a log file."""
        try:
            with open(filepath, 'a', encoding='utf-8') as log_file:
                log_file.write(f"User: {user_input}\nAI: {ai_response}\n\n")
        except UnicodeEncodeError as e:
            print(f"UnicodeEncodeError: {e}")

class TranscriptCollector:
    def __init__(self):
        """Initialize the transcript collector."""
        self.reset()

    def reset(self):
        """Reset the transcript parts."""
        self.transcript_parts = []

    def add_part(self, part):
        """Add a part to the transcript."""
        self.transcript_parts.append(part)

    def get_full_transcript(self):
        """Return the full transcript as a single string."""
        return ' '.join(self.transcript_parts)

transcript_collector = TranscriptCollector()

async def get_transcript(callback):
    """Collect transcript from the microphone using Deepgram API."""
    transcription_complete = asyncio.Event()  # Event to signal transcription completion
    try:
        config = DeepgramClientOptions(options={"keepalive": "true"})
        deepgram = DeepgramClient("", config)
        dg_connection = deepgram.listen.asyncwebsocket.v("1")
        print("Listening...")

        async def on_message(self, result, **kwargs):
            """Handle incoming transcription messages."""
            sentence = result.channel.alternatives[0].transcript
            if not result.speech_final:
                transcript_collector.add_part(sentence)
            else:
                transcript_collector.add_part(sentence)
                full_sentence = transcript_collector.get_full_transcript()
                if len(full_sentence.strip()) > 0:
                    full_sentence = full_sentence.strip()
                    print(f"Human: {full_sentence}")
                    callback(full_sentence)
                    transcript_collector.reset()
                    transcription_complete.set()  # Signal to stop transcription and exit

        dg_connection.on(LiveTranscriptionEvents.Transcript, on_message)
        options = LiveOptions(
            model="nova-2",
            punctuate=True,
            language="en-IN",
            encoding="linear16",
            channels=1,
            sample_rate=16000,
            endpointing=300,
            smart_format=True,
        )
        await dg_connection.start(options)
        microphone = Microphone(dg_connection.send)
        microphone.start()
        await transcription_complete.wait()  # Wait for the transcription to complete
        microphone.finish()
        await dg_connection.finish()
    except Exception as e:
        print(f"Could not open socket: {e}")

class ConversationManager:
    def __init__(self):
        """Initialize the conversation manager."""
        self.transcription_response = ""
        self.llm = LanguageModelProcessor()
        self.welcome_message = "I am GetScreened PrepGuru. I will guide you step by step to improve your interview skills and help you become a better version of yourself."

    async def main(self):
        """Main function to manage the conversation flow."""
        # Speak the welcome message
        text_to_speech(self.welcome_message)

        # Loop to ask for introduction up to 3 times
        for attempt in range(3):
            text_to_speech(f"Please introduce yourself. This is attempt {attempt + 1} of 3.")
            await get_transcript(self.handle_transcription)
            
            # Provide feedback on the introduction
            feedback = self.llm.provide_feedback_on_introduction(self.transcription_response)
            text_to_speech(feedback)

            # If feedback is positive, stop asking for further introductions
            if "good" in feedback.lower():
                text_to_speech("Great! Your introduction is good. Let's move on.")
                break
            elif attempt < 2:  # Only ask again if not on the last attempt
                text_to_speech("Let's try again and improve.")

        # After 3 attempts, suggest practicing more if the introduction is not satisfactory
        if "good" not in feedback.lower():
            text_to_speech("It seems like your introduction needs more practice. Please work on it and try again later.")
        
        # End of the session
        text_to_speech("Thank you for participating in the session.")

    def handle_transcription(self, transcription):
        """Handle the transcription and update the response."""
        self.transcription_response = transcription.strip()

if __name__ == "__main__":
    manager = ConversationManager()
    asyncio.run(manager.main())
 

compare file

async def ask_for_introduction(self):
    iteration_count = 0  # To count the number of attempts

    while iteration_count < 3:  # Allow for up to 3 attempts
        # Step 1: Ask for introduction
        intro_prompt = "Please introduce yourself."
        print(f"AI: {intro_prompt}")
        text_to_speech(intro_prompt, self.speech_synthesizer)

        # Capture user's introduction
        user_intro = await self.listen_for_speech()
        if user_intro:
            print(f"User: {user_intro}")

            # Step 2: Analyze user's introduction and give feedback
            feedback = self.llm.process(f"Here is an introduction: {user_intro}. Give me feedback on the introduction, what is good, and what is missing.")
            print(f"AI Feedback: {feedback}")
            text_to_speech(feedback, self.speech_synthesizer)

            # Step 3: Check if feedback is satisfactory
            if "good" in feedback.lower():
                print("Feedback was satisfactory.")
                return True  # Exit loop and move to the next stage (resume questions)
            else:
                print("Your introduction needs improvement.")
                text_to_speech("Your introduction needs improvement. Please try again.", self.speech_synthesizer)
        else:
            print("No valid introduction received. Please try again.")
            text_to_speech("No valid introduction received. Please try again.", self.speech_synthesizer)

        iteration_count += 1  # Increment the attempt count

    # After 3 attempts, provide final feedback
    if iteration_count == 3:
        practice_feedback = "You have made three attempts. Please practice your introduction more."
        print(practice_feedback)
        text_to_speech(practice_feedback, self.speech_synthesizer)

    return False  # Return false if the candidate couldn't provide a satisfactory introduction

async def main(self):
    # Start with the welcome message
    welcome_message = "I am GetScreened PrepGuru. I will guide you step by step to improve your interview skills and help you become a better version of yourself."
    print(f"AI: {welcome_message}")
    text_to_speech(welcome_message, self.speech_synthesizer)

    # Ask for the introduction
    introduction_satisfactory = await self.ask_for_introduction()

    if introduction_satisfactory:
        # Load resume and process it after satisfactory introductions
        resume_path = r"Resume\Ankush Mahore.pdf"  # Adjust this path to your resume file
        resume_text = self.llm.load_resume(resume_path)

        # Ask questions based on the resume
        await self.ask_questions_based_on_resume(resume_text)
    else:
        print("Moving on without resume questions due to unsatisfactory introduction.")


correct code

import asyncio
import os
import io
import time
import pickle
from dotenv import load_dotenv
import azure.cognitiveservices.speech as speechsdk
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
import PyPDF2


# Load environment variables
load_dotenv()

class LanguageModelProcessor:
    def __init__(self):
        self.llm = ChatGroq(
            temperature=0,
            model_name="llama-3.1-70b-versatile",
            groq_api_key=os.getenv("GROQ_API_KEY")
        )
        self.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        
        with open('system_prompt2.txt', 'r') as file:
            system_prompt = file.read().strip()
        
        self.prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            HumanMessagePromptTemplate.from_template("{text}")
        ])
        
        self.conversation = LLMChain(
            llm=self.llm,
            prompt=self.prompt,
            memory=self.memory
        )

    def process(self, text):
        self.memory.chat_memory.add_user_message(text)
        start_time = time.time()
        response = self.conversation.invoke({"text": text})
        end_time = time.time()
        self.memory.chat_memory.add_ai_message(response['text'])
        elapsed_time = int((end_time - start_time) * 1000)
        print(f"LLM ({elapsed_time}ms): {response['text']}")
        return response['text']

    def load_resume(self, resume_path):
        with open(resume_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            resume_text = ""
            for page in reader.pages:
                resume_text += page.extract_text()
        return resume_text.strip()

    def save_memory(self, filepath='conversation_memory.pkl'):
        with open(filepath, 'wb') as f:
            pickle.dump(self.memory, f)
        print(f"Memory saved to {filepath}")

    def load_memory(self, filepath='conversation_memory.pkl'):
        if os.path.exists(filepath):
            with open(filepath, 'rb') as f:
                self.memory = pickle.load(f)
            print(f"Memory loaded from {filepath}")
        else:
            print(f"No memory file found at {filepath}, starting fresh.")

    def save_conversation(self, user_input, ai_response, filepath='conversation_log.txt'):
        try:
            with open(filepath, 'a', encoding='utf-8') as log_file:
                log_file.write(f"User: {user_input}\nAI: {ai_response}\n\n")
        except UnicodeEncodeError as e:
            print(f"UnicodeEncodeError: {e}")

    def load_conversation(self, filepath='conversation_log.txt'):
        if os.path.exists(filepath):
            with open(filepath, 'r', encoding='utf-8') as log_file:
                previous_conversation = log_file.read()
            print("Previous conversation loaded:\n", previous_conversation)
            return True
        else:
            print("Unable to load conversation_log.txt. I don't have your last information available.")
            return False


def setup_azure_speech(api_key, region):
    speech_config = speechsdk.SpeechConfig(subscription=api_key, region=region)
    speech_config.speech_recognition_language = "en-IN"
    speech_config.set_property(speechsdk.PropertyId.SpeechServiceConnection_InitialSilenceTimeoutMs, '5000')
    speech_config.set_property(speechsdk.PropertyId.SpeechServiceConnection_EndSilenceTimeoutMs, '4000')

    audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)
    return speech_recognizer


def setup_azure_tts(api_key, region):
    speech_config = speechsdk.SpeechConfig(subscription=api_key, region=region)
    audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)
    speech_config.speech_synthesis_voice_name = 'en-IN-AaravNeural'
    speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)
    return speech_synthesizer


def text_to_speech(text, speech_synthesizer):
    ssml_text = f"""
    <speak version='1.0' xmlns='http://www.w3.org/2001/10/synthesis' xml:lang='en-IN'>
        <voice name='en-IN-AaravNeural'>
            <prosody rate='medium'>
                {text}
            </prosody>
        </voice>
    </speak>
    """
    try:
        speech_synthesis_result = speech_synthesizer.speak_ssml_async(ssml_text).get()
        if speech_synthesis_result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
            print("Speech synthesis complete")
        else:
            print(f"Speech synthesis failed: {speech_synthesis_result.reason}")
    except Exception as e:
        print(f"An error occurred during speech synthesis: {e}")


class ConversationManager:
    def __init__(self):
        self.llm = LanguageModelProcessor()
        self.api_key = os.getenv("AZURE_API_KEY")
        self.region = os.getenv("AZURE_REGION")
        self.speech_recognizer = setup_azure_speech(self.api_key, self.region)
        self.speech_synthesizer = setup_azure_tts(self.api_key, self.region)
        self.is_running = True

    async def listen_for_speech(self):
        print("Listening...")
        result = await self.recognize_speech()
        if result.reason == speechsdk.ResultReason.RecognizedSpeech:
            return result.text
        elif result.reason == speechsdk.ResultReason.NoMatch:
            print("No speech could be recognized")
        return None

    async def recognize_speech(self):
        loop = asyncio.get_running_loop()
        future = loop.create_future()

        def recognized_cb(evt):
            if not future.done():
                loop.call_soon_threadsafe(future.set_result, evt.result)

        def canceled_cb(evt):
            if not future.done():
                loop.call_soon_threadsafe(future.set_exception, Exception(f"Speech recognition canceled: {evt.reason}"))

        self.speech_recognizer.recognized.connect(recognized_cb)
        self.speech_recognizer.canceled.connect(canceled_cb)

        await loop.run_in_executor(None, self.speech_recognizer.start_continuous_recognition)

        try:
            result = await asyncio.wait_for(future, timeout=180.0)  # 180 second timeout
        except asyncio.TimeoutError:
            print("Speech recognition timed out")
            result = None
        finally:
            await loop.run_in_executor(None, self.speech_recognizer.stop_continuous_recognition)

        return result
    
    #####################################################################################################
    # async def ask_for_introduction(self):
    #     iteration_count = 0
    #     feedback = ""

    #     while "good" not in feedback.lower() and iteration_count < 2:
    #         # First introduction
    #         intro_prompt = "Please introduce yourself."
    #         print(f"AI: {intro_prompt}")
    #         text_to_speech(intro_prompt, self.speech_synthesizer)

    #         user_intro = await self.listen_for_speech()
    #         if user_intro:
    #             print(f"User: {user_intro}")
    #             feedback = self.llm.process(user_intro)
    #             print(f"AI: {feedback}")
    #             text_to_speech(feedback, self.speech_synthesizer)

    #         iteration_count += 1

    ######################################################################################################

    # async def ask_for_introduction(self):
    #     iteration_count = 0  # To count the number of attempts

    #     while iteration_count < 3:  # Allow for up to 3 attempts
    #         # Step 1: Ask for introduction
    #         intro_prompt = "Please introduce yourself."
    #         print(f"AI: {intro_prompt}")
    #         text_to_speech(intro_prompt, self.speech_synthesizer)

    #         # Capture user's introduction
    #         user_intro = await self.listen_for_speech()
    #         if user_intro:
    #             print(f"User: {user_intro}")

    #             # Step 2: Analyze user's introduction and give feedback
    #             feedback = self.llm.process(f"Here is an introduction: {user_intro}. Give me feedback on the introduction, what is good, and what is missing.")
    #             print(f"AI Feedback: {feedback}")
    #             text_to_speech(feedback, self.speech_synthesizer)

    #             # Step 3: Check if feedback is satisfactory
    #             if "good" in feedback.lower():
    #                 print("Feedback was satisfactory.")
    #                 break  # Exit loop if feedback is satisfactory
    #             else:
    #                 print("Your introduction needs improvement.")
    #                 text_to_speech("Your introduction needs improvement. Please try again.", self.speech_synthesizer)
    #         else:
    #             print("No valid introduction received. Please try again.")
    #             text_to_speech("No valid introduction received. Please try again.", self.speech_synthesizer)

    #         iteration_count += 1  # Increment the attempt count

    #     # After 3 attempts, provide final feedback
    #     if iteration_count == 3:
    #         practice_feedback = "You have made three attempts. Please practice your introduction more."
    #         print(practice_feedback)
    #         text_to_speech(practice_feedback, self.speech_synthesizer)
    #     # Optionally, you could restart asking for introduction or move to the next stage
    #     # Uncomment the following line if you want to ask again after three attempts
    #     # await self.ask_for_introduction()  # Restart the process if needed



    async def ask_for_introduction(self):
        iteration_count = 0  # To count the number of attempts

        while iteration_count < 3:  # Allow for up to 3 attempts
            # Step 1: Ask for introduction
            intro_prompt = "Please introduce yourself."
            print(f"AI: {intro_prompt}")
            text_to_speech(intro_prompt, self.speech_synthesizer)

            # Capture user's introduction
            user_intro = await self.listen_for_speech()
            if user_intro:
                print(f"User: {user_intro}")

                # Step 2: Analyze user's introduction and give feedback
                feedback = self.llm.process(f"Here is an introduction: {user_intro}. Give me feedback on the introduction, what is good, and what is missing.")
                print(f"AI Feedback: {feedback}")
                text_to_speech(feedback, self.speech_synthesizer)

                # Step 3: Check if feedback is satisfactory
                if "good" in feedback.lower():
                    print("Feedback was satisfactory.")
                    return True  # Exit loop and move to the next stage (resume questions)
                else:
                    print("Your introduction needs improvement.")
                    text_to_speech("Your introduction needs improvement. Please try again.", self.speech_synthesizer)
            else:
                print("No valid introduction received. Please try again.")
                text_to_speech("No valid introduction received. Please try again.", self.speech_synthesizer)

            iteration_count += 1  # Increment the attempt count

        # After 3 attempts, provide final feedback
        if iteration_count == 3:
            practice_feedback = "You have made three attempts. Please practice your introduction more."
            print(practice_feedback)
            text_to_speech(practice_feedback, self.speech_synthesizer)

        return False  # Return false if the candidate couldn't provide a satisfactory introduction

###############################################################
        # Optionally, you could restart asking for introduction or move to the next stage
        # Uncomment the following line if you want to ask again after three attempts
        # await self.ask_for_introduction()  # Restart the process if needed



        # After 2 iterations, suggest practice and move on
        # if "good" not in feedback.lower():
        #     text_to_speech("Please practice this introduction more. Let's move on to the next question.", self.speech_synthesizer)


    async def ask_questions_based_on_resume(self, resume_text):
        questions_prompt = f"Based on the following resume, please ask relevant questions: {resume_text}"
        questions = self.llm.process(questions_prompt)
        print(f"AI: {questions}")
        text_to_speech(questions, self.speech_synthesizer)

        while True:
            user_response = await self.listen_for_speech()
            if user_response:
                print(f"User: {user_response}")
                feedback = self.llm.process(user_response)
                print(f"AI: {feedback}")
                text_to_speech(feedback, self.speech_synthesizer)

    async def main(self):
        # Start with the welcome message
        welcome_message = "I am GetScreened PrepGuru. I will guide you step by step to improve your interview skills and help you become a better version of yourself."
        print(f"AI: {welcome_message}")
        text_to_speech(welcome_message, self.speech_synthesizer)

        # Ask for the introduction twice
        await self.ask_for_introduction()

        # Load resume and process it after introductions
        resume_path = r"Resume\Abhinav Anand Resume.pdf"  # Adjust this path to your resume file  
        resume_text = self.llm.load_resume(resume_path)

        # Ask questions based on the resume
        await self.ask_questions_based_on_resume(resume_text)

if __name__ == "__main__":
    manager = ConversationManager()
    asyncio.run(manager.main())


# download resume

In [1]:
import requests
import json
import os

# Configuration
API_ENDPOINT = "https://sy5r6puueraf3gty6ujp5iajmi.appsync-api.ap-south-1.amazonaws.com/graphql"
API_KEY = "da2-c3tlmtun7vcz5e2t7mcfizxma4"
RESUME_FOLDER = "resumes"  # Folder to save resumes

# Headers for the request
headers = {
    "x-api-key": API_KEY,
    "Content-Type": "application/json"
}

# GraphQL query for listing candidate profiles with pagination
list_candidate_profiles_query = """
query ListCandidateProfiles(
  $filter: ModelCandidateProfileFilterInput
  $limit: Int
  $nextToken: String
) {
  listCandidateProfiles(
    filter: $filter
    limit: $limit
    nextToken: $nextToken
  ) {
    items {
      id
      fullName
      email
      resumeURL
    }
    nextToken
  }
}
"""

# Function to download a file from a URL and save it to a folder
def download_resume(resume_url, full_name):
    # Create the folder if it doesn't exist
    if not os.path.exists(RESUME_FOLDER):
        os.makedirs(RESUME_FOLDER)

    # Determine the file name
    resume_file_name = f"{full_name.replace(' ', '_')}_resume.pdf"
    resume_path = os.path.join(RESUME_FOLDER, resume_file_name)

    # Download the resume
    response = requests.get(resume_url)

    # Save the resume to the folder
    with open(resume_path, 'wb') as file:
        file.write(response.content)

    print(f"Resume downloaded and saved as {resume_path}")

# Fetch candidate profiles and download their resumes
def fetch_candidate_profiles(email, limit=10):
    profiles = []
    next_token = None

    while True:
        # Payload with the query, variables, and filter for the specific email
        payload = {
            "query": list_candidate_profiles_query,
            "variables": {
                "filter": {
                    "email": {
                        "eq": email
                    }
                },
                "limit": limit,
                "nextToken": next_token
            }
        }

        # Make the request to the GraphQL API
        response = requests.post(API_ENDPOINT, headers=headers, json=payload)

        # Parse the JSON response
        data = response.json()

        # Extract the candidate profiles and nextToken for pagination
        result = data.get('data', {}).get('listCandidateProfiles', {})
        profiles.extend(result.get('items', []))
        next_token = result.get('nextToken')

        # If there's no more data to fetch, exit the loop
        if not next_token:
            break

    # Download the resumes
    for profile in profiles:
        full_name = profile.get('fullName', 'Unknown')
        email = profile.get('email', 'Unknown')
        resume_url = profile.get('resumeURL', None)

        print(f"ID: {profile['id']}")
        print(f"Full Name: {full_name}")
        print(f"Email: {email}")
        print(f"Resume URL: {resume_url}")

        # Download the resume if the URL exists
        if resume_url:
            download_resume(resume_url, full_name)
        else:
            print(f"No resume available for {full_name}")
        
        print("-" * 50)

# Fetch and download resumes for the specified email with pagination
fetch_candidate_profiles('sushrutnistane097@gmail.com')


ID: c3c58509-b4b6-4f54-932a-6c6951d87ef0
Full Name: Sushrut Rajeshwar Nistane
Email: sushrutnistane097@gmail.com
Resume URL: https://getscreenedstoragebucketf24ed-devget.s3.amazonaws.com/public/1724521431224_Sushrut Nistane Resume  pdf (3).pdf
Resume downloaded and saved as resumes\Sushrut_Rajeshwar_Nistane_resume.pdf
--------------------------------------------------
